Arrancamos Apche Nifi, hay dos maneras: con el comando "start" o "run". El comando "run" se queda con el control del shell, "start" arranca Nifi en background y nos deja el control de la terminal.
* `nifi-1.16.2/bin/nifi.sh start`
* Para comprobar que haya arrancado correctamente uso el comando `jps`
* https://localhost:8443/nifi Ahora el puerto de default de Nifi es 8443
* En el fichero logs/nifi-app.log puedo encontrar el usuario y la contraseña (Generated Username [USERNAME] y Generated Password [PASSWORD])

##### Ejemplo 1

* Empiezo creando dos carpetas: "nifisource" y "nifitarget"
* Creo dos procesadores en Nifi. El primer procesador es `GetFile`, tengo que modificar el directorio de entrada ("Input Directory") insertando la ruta a mi carpeta "nifisource"
* El segundo procesador es `PutFile`, también tengo que configurarlo con el directorio de entrada (ruta a "nifitarget")
* Puedo cambiar los nombres de los procesadores (ejemplo: "Leer data desde fuente" y "Volcar data destino")
* En el procesador de destino tengo que seleccionar una opción en "relationships" > "Automatically terminate/retry relationships"
* Conecto los dos procesadores
* Creo dos ficheros nuevos en la carpeta "nifisource" `echo "Hola mundo" > file1.txt` y `echo "Hola mundo 2" > file2.txt`
* Los componentes se pueden arrancar por partes, empiezo con el procesador GetFile y sigo con PutFile
* Los ficheros ya se han volcado en la carpeta "nifitarget"

##### Ejemplo 2

* Empiezo creando en Nifi un procesador GetFile como en el ejemplo 1 y un segundo procesador `PutHDFS` donde tengo que insertar una ruta de destino (Properties > Directory): `/user/spark/nifi/hdfsexample1`
* Arranco HDFS: `start-dfs.sh` y compruebo que todo esté bien con el comando `jps`
* Compruebo que existe la ruta `/user/spark/nifi` y si no la creo. El directorio "hdfsexample1" se crea solo al momento de volcar los datos
* En el procesador de destino tengo que seleccionar una opción en "relationships" > "Automatically terminate/retry relationships"
* En el procesador de destino también tengo que configurar "properties" > "Hadoop configuration resources", tengo que añadir las rutas: `/home/spark/Escritorio/hadoop/etc/hadoop/core-site.xml, /home/spark/Escritorio/hadoop/etc/hadoop/hdfs-site.xml`
* Creo dos ficheros nuevos en la carpeta "nifisource" `echo "Hola mundo" > file1.txt` y `echo "Hola mundo 2" > file2.txt`
* Arranco el primer procesador y luego el segundo también
* Los ficheros ya no están en la carpeta "nifisource"
* Compruebo si en hdfs se ha creado la carpeta hdfsexample1 en la ruta /user/spark/nifi/hdfsexample1
* `spark@spark-virtualBox:~/Escritorio$ hdfs dfs -ls nifi/hdfsexample1`
`Found 4 items`
`-rw-r--r--   1 spark supergroup         11 2022-06-06 13:52 nifi/hdfsexample1/file1.txt`
`-rw-r--r--   1 spark supergroup         13 2022-06-06 13:52 nifi/hdfsexample1/file2.txt`
`-rw-r--r--   1 spark supergroup         13 2022-06-06 13:53 nifi/hdfsexample1/file3.txt`
`-rw-r--r--   1 spark supergroup         13 2022-06-06 13:54 nifi/hdfsexample1/file4.txt`
* Para comprobar el contenido de todos los ficheros: `hdfs dfs -cat /user/spark/nifi/hdfsexample1/*`

##### Ejemplo 3

* Empiezo creando el primer procesador GetFile como en los otros ejemplos
* Añado un segundo procesador `UpdateAttribute` donde tenemos que añadir una nueva propiedad que llamamos "ts" (timestamp) y el contenido será: ${now()}
* En la documentación de Nifi --> Expression Language Guide > Date Manipulation (https://nifi.apache.org/docs/nifi-docs/). Aquí encuentro el método now, una representación de la fecha en este momento. 
* Añado un nuevo procesador "PutHDFS" (cómo en el segundo ejemplo) y aquí tengo que cambiar la ruta donde se van a volcar los datos: `/user/spark/nifi/hdfsexample2/${ts:format("yyyy-MM-dd")}/${ts:format("HHmm")}` 
* Conecto los tres procesadores
* Voy a crear unos ficheros en la carpeta "nifisource" (`echo "Hola Mundo" > file1.txt` `echo "Hola Mundo 2" > file2.txt`)
* En HDFS ya tengo creada la carpeta "hdfsexample2" y dentro las carpetas se han creado con fechas y horas--> `hdfs dfs -ls -R /user/spark/nifi/hdfsexample2`
`drwxr-xr-x   - spark supergroup          0 2022-06-06 16:09 /user/spark/nifi/hdfsexample2/2022-06-06`
`drwxr-xr-x   - spark supergroup          0 2022-06-06 16:08 /user/spark/nifi/hdfsexample2/2022-06-06/1608`
`-rw-r--r--   1 spark supergroup         13 2022-06-06 16:08 /user/spark/nifi/hdfsexample2/2022-06-06/1608/file1.txt`
`-rw-r--r--   1 spark supergroup         13 2022-06-06 16:08 /user/spark/nifi/hdfsexample2/2022-06-06/1608/file2.txt`
`drwxr-xr-x   - spark supergroup          0 2022-06-06 16:09 /user/spark/nifi/hdfsexample2/2022-06-06/1609`
`-rw-r--r--   1 spark supergroup         13 2022-06-06 16:09 /user/spark/nifi/hdfsexample2/2022-06-06/1609/file3.txt`